In [20]:
import os
import sys
import openai
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

In [ ]:
current_dir = os.path.dirname(os.path.abspath("__file__"))  
parent_dir = os.path.abspath(os.path.join(current_dir, "..", ".."))  
sys.path.append(parent_dir)

from utils.load_function_schemas import load_function_schemas

NameError: name 'os' is not defined

In [22]:
from agents import (
    InitialGeneratorAgent,
    ReviewerAgent,
    ChallengerAgent,
    RefinerAgent,
    DeciderAgent
)

ModuleNotFoundError: No module named 'base_agent'

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY") 

NameError: name 'load_dotenv' is not defined